In [1]:
import torch

In [4]:
x = torch.randn(2, 5, 6)  # Example input tensor

In [5]:
x

tensor([[[ 1.0216,  0.7342, -0.6790, -0.2337, -0.2148,  1.6775],
         [ 1.0349,  0.3573, -1.1944, -0.2279, -0.6715, -0.4217],
         [-0.4813, -0.2808, -0.1857, -0.8592, -1.1012, -1.0274],
         [ 0.4788, -1.0373,  1.3541, -0.4275, -0.6349, -0.1834],
         [-1.0881,  1.2217,  0.2649,  1.0102, -0.5774, -0.8120]],

        [[ 0.1743, -0.3477, -0.4250, -1.1201, -0.7737, -0.0672],
         [-0.6176,  0.6850, -1.1081, -0.8190,  1.2918,  0.5051],
         [ 0.2261,  0.1991,  0.8351,  1.1934, -0.2743,  1.3275],
         [-1.6223, -1.0311,  0.2556, -0.1470,  0.2200,  0.1023],
         [-0.3080, -0.6187,  0.2693, -0.8978,  0.8426,  0.9483]]])

In [6]:
x.view(2, 5, 3, 2)  # Reshape to (batch_size, seq_len, d_model//2, 2)

tensor([[[[ 1.0216,  0.7342],
          [-0.6790, -0.2337],
          [-0.2148,  1.6775]],

         [[ 1.0349,  0.3573],
          [-1.1944, -0.2279],
          [-0.6715, -0.4217]],

         [[-0.4813, -0.2808],
          [-0.1857, -0.8592],
          [-1.1012, -1.0274]],

         [[ 0.4788, -1.0373],
          [ 1.3541, -0.4275],
          [-0.6349, -0.1834]],

         [[-1.0881,  1.2217],
          [ 0.2649,  1.0102],
          [-0.5774, -0.8120]]],


        [[[ 0.1743, -0.3477],
          [-0.4250, -1.1201],
          [-0.7737, -0.0672]],

         [[-0.6176,  0.6850],
          [-1.1081, -0.8190],
          [ 1.2918,  0.5051]],

         [[ 0.2261,  0.1991],
          [ 0.8351,  1.1934],
          [-0.2743,  1.3275]],

         [[-1.6223, -1.0311],
          [ 0.2556, -0.1470],
          [ 0.2200,  0.1023]],

         [[-0.3080, -0.6187],
          [ 0.2693, -0.8978],
          [ 0.8426,  0.9483]]]])